In [ ]:
#default_exp XResNet

In [ ]:
#export
from nbdev.showdoc import show_doc
from ModernArchitecturesFromPyTorch.nb_ScratchToPytorch import *

# XResNet with PyTorch
> Implementing the second generation ResNet with some tweaks presented in: https://arxiv.org/abs/1812.01187

## ImageNet Dataset

I think we've been using MNIST for a little too long, to really test out the models going forward we're going to need to start using some bigger and harder datasets. Opting for another class, let's get Imagenet setup using FastAI's DataBlock API

In [ ]:
#export
def get_imaggenette():
    "Grabs imagenette dataset"
    path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)
    tfms = get_transforms()
    return (ImageList.from_folder(path).split_by_rand_pct(0.2).label_from_folder().transform(tfms, size=128).databunch(bs=16))

## Fundamental ResNet Modules

In [ ]:
#export
class Identity(nn.Module):
    "Layer that doesn't do anything"
    def __init__(self): super().__init__()
    def forward(self, xb): return xb

In [ ]:
#export
class AutoConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        "Automatically resizing convolution so output feature map size is same as input"
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, **kwargs)
    
    def forward(self, xb): return self.conv(xb)

In [ ]:
#export
class ConvBatchLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, Activation=Identity, **kwargs):
        "Convolutional layer with normalization and non linearity"
        super().__init__()
        self.conv = AutoConv(in_channels, out_channels, kernel_size, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = Activation()
    
    def forward(self, xb): return self.activation(self.bn(self.conv(xb)))

In [ ]:
#export
class BaseRes(nn.Module):
    def __init__(self, expansion, n_in, n_h, stride=1, Activation=nn.ReLU, **kwargs):
        "Base class for a residual layer, can be implemented as bottleneck or normal"
        super().__init__()

        n_out, n_in = n_h*expansion, n_in*expansion

        self.blocks = nn.Sequential(
            ConvBatchLayer(n_in, n_h, kernel_size=1, stride=1),
            Activation(),
            Conv(n_h, n_h, kernel_size=3, stride=stride),
            Activation(),
            ConvBatchLayer(n_h, n_out, kernel_size=1, stride=1)
        ) if expansion != 1 else nn.Sequential(
                ConvBatchLayer(n_in, n_h, kernel_size=3, stride=stride),
                Activation(),
                ConvBatchLayer(n_h, n_out, kernel_size=3, stride=1),
            )
        
        self.identity = Identity() if n_in == n_out else nn.Sequential(
                    nn.AvgPool2d(kernel_size=2),
                    ConvBatchLayer(n_in, n_out, stride=1, kernel_size=1)
                )
        
        self.activation = Activation()

    def forward(self, xb):
        return self.activation(self.identity(xb) + self.blocks(xb))

In [ ]:
#export
class XResnet(nn.Module):
    def __init__(self, expansion, c_in, c_out, layer_depths, **kwargs):
        "XResNet in it's full glory"
        super().__init__()

        gate_depths = [c_in, (c_in+1)*8, 64, 64]
        gate_sizes = list(zip(gate_depths, gate_depths[1:]))

        gate = [ConvBatchLayer(in_c, out_c, stride=1 if i != 0 else 2) for
                i, (in_c, out_c) in enumerate(gate_sizes)]
        gate.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

        body_sizes = [64//expansion,64,128,256,512]
        res_layers = [self.sequential_res_blocks(expansion, body_sizes[i], body_sizes[i+1],
                                      n_block=l, stride=1 if i==0 else 2, **kwargs)
                  for i,l in enumerate(layer_depths)]
        
        decoder = [
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(body_sizes[-1]*expansion, c_out),
        ]

        self.resnet = nn.Sequential(
            *gate,
            *res_layers,
            *decoder
        )
    
    @classmethod
    def sequential_res_blocks(self, expansion, ni, nf, n_block, stride, **kwargs):
        return nn.Sequential(
            *[BaseRes(expansion, ni if i==0 else nf, nf, stride if i==0 else 1, **kwargs) for i in range(n_block)]
        )
    
    def forward(self, xb): return self.resnet(xb)

In [ ]:
#export
def getResnet(size, c_in, c_out, **kwargs):
    if size == 18: return XResnet(1, c_in, c_out, [2, 2,  2, 2], **kwargs)
    elif size == 34: return XResnet(1, c_in, c_out, [3, 4,  6, 3], **kwargs)
    elif size == 50: return XResnet(4, c_in, c_out, [3, 4,  6, 3], **kwargs)
    elif size == 150: return XResnet(4, c_in, c_out, [3, 4, 23, 3], **kwargs)
    elif size == 152: return XResnet(4, c_in, c_out, [3, 8, 36, 3], **kwargs)

In [ ]:
#export
class CudaCallback(Callback):
    "Quick callback to put model onto the GPU"
    def before_fit(self): self.model.cuda()

In [ ]:
run = get_runner(get_learner(getResnet(18, 3, 10)), [ProgressCallback(), Stats([accuracy]), CudaCallback()])

In [ ]:
run.fit(1, 1e-3)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.731658,0.417290,1.382680,0.539955,01:38


In [ ]:
!python notebook2script.py XResNet.ipynb

Converted XResNet.ipynb to ModernArchitecturesFromPyTorch/nb_XResNet.py
